# Run this code to deploy an agent

In [64]:
import vertexai
from virtual_nurse.agent import root_agent

PROJECT_ID = "virtual-nurse-450613"
LOCATION = "us-central1"
STAGING_BUCKET = "gs://deploy-virtual-nurse"

vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

from vertexai.preview import reasoning_engines

app = reasoning_engines.AdkApp(
    agent=root_agent,
    enable_tracing=True,
)

session = app.create_session(user_id="u_123")

In [65]:
session

Session(id='796f3029-ae2c-442a-8adc-5ea7e498ab7e', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1750463830.4999225)

In [66]:
app.list_sessions(user_id="u_123")

ListSessionsResponse(sessions=[Session(id='796f3029-ae2c-442a-8adc-5ea7e498ab7e', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1750463830.4999225)])

In [67]:
for event in app.stream_query(
    user_id="u_123",
    session_id=session.id,
    message="Scarlett, age 10, had a tonsillectomy 5 days ago and is currently visiting cousins out of town. The parent reports that the son's mouth smells really bad and he says it tastes gross. The patient does not report bleeding. The distance from the surgical hospital is 45 minutes and from the nearest hospital is 20 minutes. The patient last ate this afternoon and drank 1.5 cups of fluids in the past day. Lips are not chapped. The patient is not acting like themselves\u2014complaining a lot. Pain is not limiting intake. The patient is not experiencing nausea or vomiting and has no fevers. Pain is reported in the throat. There is no ear pain. The patient has a sore throat. Pain is managed with ibuprofen. Medications are not being alternated and they have not run out. Pain is not impacting intake. There is no bloody sputum. The throat has white patches only. The tongue/mouth has a yellowish coating. There is no breathing history and no cyanosis. The patient is annoyed and avoidant and not acting normal.",
):
    print(event)

{'content': {'parts': [{'text': "Hello! I'm here to help you with Scarlett. I need to ask you a few questions to understand the situation better.\n\nWhat surgery did Scarlett have?"}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 33, 'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 33}], 'prompt_token_count': 1973, 'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 1973}], 'thoughts_token_count': 1017, 'total_token_count': 3023, 'traffic_type': <TrafficType.ON_DEMAND: 'ON_DEMAND'>}, 'invocation_id': 'e-3d71a35b-5e38-43b0-b76e-4fdaabf2f7be', 'author': 'triage_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'T57SwpbP', 'timestamp': 1750463842.248229}


In [68]:
from vertexai import agent_engines

remote_app = agent_engines.create(
    agent_engine=root_agent,
    requirements=[
        "google-cloud-aiplatform[adk,agent_engines]",  
    ],
    extra_packages=[
        "virtual_nurse",  
    ]
)

Deploying google.adk.agents.Agent as an application.
Identified the following requirements: {'pydantic': '2.11.7', 'google-cloud-aiplatform': '1.97.0', 'cloudpickle': '3.1.1'}
The following requirements are missing: {'pydantic', 'cloudpickle'}
The following requirements are appended: {'pydantic==2.11.7', 'cloudpickle==3.1.1'}
The final list of requirements: ['google-cloud-aiplatform[adk,agent_engines]', 'pydantic==2.11.7', 'cloudpickle==3.1.1']
Using bucket deploy-virtual-nurse
Wrote to gs://deploy-virtual-nurse/agent_engine/agent_engine.pkl
Writing to gs://deploy-virtual-nurse/agent_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://deploy-virtual-nurse/agent_engine/dependencies.tar.gz
Creating AgentEngine
Create AgentEngine backing LRO: projects/533873564462/locations/us-central1/reasoningEngines/8665884457200254976/operations/469877221843009536
View progress and logs at https://console.cloud.google.com/logs/query?project=virtual-nurse-450613
AgentEn

In [69]:
remote_app.resource_name

'projects/533873564462/locations/us-central1/reasoningEngines/8665884457200254976'

In [70]:
remote_session = remote_app.create_session(user_id="u_123")
remote_session

{'appName': '8665884457200254976',
 'lastUpdateTime': 1750464115.200907,
 'userId': 'u_123',
 'state': {},
 'events': [],
 'id': '4254349687202512896'}

In [72]:
for event in remote_app.stream_query(
    user_id="u_123",
    session_id=remote_session["id"],
    message="Scarlett, age 10, had a tonsillectomy 5 days ago and is currently visiting cousins out of town. The parent reports that the son's mouth smells really bad and he says it tastes gross. The patient does not report bleeding. The distance from the surgical hospital is 45 minutes and from the nearest hospital is 20 minutes. The patient last ate this afternoon and drank 1.5 cups of fluids in the past day. Lips are not chapped. The patient is not acting like themselves\u2014complaining a lot. Pain is not limiting intake. The patient is not experiencing nausea or vomiting and has no fevers. Pain is reported in the throat. There is no ear pain. The patient has a sore throat. Pain is managed with ibuprofen. Medications are not being alternated and they have not run out. Pain is not impacting intake. There is no bloody sputum. The throat has white patches only. The tongue/mouth has a yellowish coating. There is no breathing history and no cyanosis. The patient is annoyed and avoidant and not acting normal.",
):
    print(event)

{'content': {'parts': [{'text': "Here is a summary of the information you provided:\n\nPatient name: Scarlett\nPatient age: 10 years old\nSurgery type: Tonsillectomy\nWhen surgery happened: 5 days ago\nCurrent location: Out of town, visiting cousins\nBleeding reported: No\nBreathing OK: Yes\nDistance from surgical hospital: 45 minutes\nDistance from the nearest hospital: 20 minutes\nWhen the patient last ate: This afternoon\nDrink amount in the past day: 1.5 cups\nLips chapped/appearance: Lips are not chapped.\nActing like themselves or not: Not acting like themselves—complaining a lot.\nPain is preventing them from eating and drinking: No\nNausea or vomiting: No\nFever or not: No fever\nPain or not: Yes\nIf pain, pain location: Throat\nIf pain, pain management regimen: Ibuprofen\nBloody sputum or not: No\nHealing issues or not: Yes\nIf there is healing issues, how the throat looks: White patches only.\nIf there is healing issues, what else you see: Yellowish coating on tongue/mouth.\n